<a href="https://colab.research.google.com/github/JayeonKangNature/Bachelor_Project_Thesis_2023/blob/main/Faster_R_CNN_train_medical_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For mouting
from google.colab import drive
import torch
import os
import json

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import os
import torch
import torch.utils.data
import torchvision
from PIL import Image
from pycocotools.coco import COCO


class myOwnDataset(torch.utils.data.Dataset):
    def __init__(self, root, annotation, transforms=None):
        self.root = root
        self.transforms = transforms
        self.coco = COCO(annotation)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index):
        # Own coco file
        coco = self.coco
        # Image ID
        img_id = self.ids[index-1]
        # List: get annotation id from coco
        #ann_ids = coco.getAnnIds(imgIds=img_id)
        # Dictionary: target coco_annotation file for an image
        coco_annotation = coco.imgToAnns[img_id]
        # path for input image
        path = coco.loadImgs(img_id)[0]['file_name']
        # open the input image
        img = Image.open(os.path.join(self.root, path))

        # number of objects in the image
        num_objs = len(coco_annotation)

        # Bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, width, height]
        # In pytorch, the input should be [xmin, ymin, xmax, ymax]
        boxes = []
        for i in range(num_objs):
            xmin = coco_annotation[i]['bbox'][0]
            ymin = coco_annotation[i]['bbox'][1]
            xmax = xmin + coco_annotation[i]['bbox'][2]
            ymax = ymin + coco_annotation[i]['bbox'][3]
            boxes.append([xmin, ymin, xmax, ymax])
        print('image_id:',img_id,boxes)
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        # Tensorise img_id
        img_id = torch.tensor([img_id])
        # Size of bbox (Rectangular)
        areas = []
        for i in range(num_objs):
            areas.append(coco_annotation[i]['area'])
        areas = torch.as_tensor(areas, dtype=torch.float32)
        # Iscrowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Annotation is in dictionary format
        my_annotation = {}
        my_annotation["boxes"] = boxes
        my_annotation["labels"] = labels
        my_annotation["image_id"] = img_id
        my_annotation["area"] = areas
        my_annotation["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, my_annotation

    def __len__(self):
        return len(self.ids)

In [ ]:

# In my case, just added ToTensor
def get_transform():
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(custom_transforms)

In [ ]:
# create own Dataset
my_dataset = myOwnDataset(root='/content/drive/MyDrive/BSc_Project/Train_700/images',
                          annotation='/content/drive/MyDrive/BSc_Project/Kvasir-SEG/annotation/Train_700.json',
                          transforms=get_transform()
                          )

loading annotations into memory...
Done (t=2.85s)
creating index...
index created!


In [ ]:
my_dataset_val = myOwnDataset(root='/content/drive/MyDrive/BSc_Project/Val_300/images',annotation='/content/drive/MyDrive/BSc_Project/Kvasir-SEG/annotation/Val_300.json',transforms=get_transform())

loading annotations into memory...
Done (t=0.73s)
creating index...
index created!


In [ ]:
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [ ]:
'''
img, my_annotation, boxes_60 = my_dataset[650]

# Convert tensor to PIL Image
image_pil = transforms.ToPILImage()(img)
# Create figure and axes
fig, ax = plt.subplots(1)

# Display the image
ax.imshow(image_pil)

# Add bounding boxes to the image
x_min, y_min, x_max, y_max = boxes_60[0]
print(boxes_60[0])
width = x_max - x_min
height = y_max - y_min
rect = Rectangle((x_min, y_min), width, height, fill=False, edgecolor='red')
ax.add_patch(rect)

# Show the image with bounding boxes
plt.show()
'''

"\nimg, my_annotation, boxes_60 = my_dataset[650]\n\n# Convert tensor to PIL Image\nimage_pil = transforms.ToPILImage()(img)\n# Create figure and axes\nfig, ax = plt.subplots(1)\n\n# Display the image\nax.imshow(image_pil)\n\n# Add bounding boxes to the image\nx_min, y_min, x_max, y_max = boxes_60[0]\nprint(boxes_60[0])\nwidth = x_max - x_min\nheight = y_max - y_min\nrect = Rectangle((x_min, y_min), width, height, fill=False, edgecolor='red')\nax.add_patch(rect)\n\n# Show the image with bounding boxes\nplt.show()\n"

In [ ]:
import json
with open('/content/drive/MyDrive/BSc_Project/Kvasir-SEG/annotation/Train_700.json','r') as f:
  coco_json = json.load(f)

print(coco_json['annotations'][0])
print(coco_json['annotations'][699])


{'id': 1, 'image_id': 1, 'category_id': 1, 'bbox': [38, 5, 392, 333], 'area': 130536, 'iscrowd': 0}
{'id': 1, 'image_id': 646, 'category_id': 1, 'bbox': [0, 59, 405, 391], 'area': 158355, 'iscrowd': 0}


In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
# Batch size
train_batch_size = 8

data_loader = torch.utils.data.DataLoader(my_dataset,
                                          batch_size=train_batch_size,
                                          shuffle=True,
                                          num_workers=4,
                                          collate_fn=collate_fn)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
val_loader = torch.utils.data.DataLoader(my_dataset_val,
                                          batch_size=train_batch_size,
                                          shuffle=True,
                                          num_workers=4,
                                          collate_fn=collate_fn)

In [ ]:
# select device (whether GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# DataLoader is iterable over Dataset
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)

image_id: 682 [[312, 252, 519, 544]]
image_id: 253 [[228, 237, 483, 524]]
image_id: 483 [[1, 11, 553, 512]]
image_id: 347 [[248, 125, 571, 526]]
image_id: 640 [[204, 210, 450, 478]]
image_id: 414 [[423, 82, 510, 221]]
image_id: 348 [[133, 216, 399, 488]]
image_id: 150 [[355, 116, 593, 308]]
image_id: 63 [[288, 169, 459, 388], [229, 424, 421, 547]]
image_id: 592 [[272, 191, 471, 423]]
image_id: 570 [[108, 123, 302, 357]]
image_id: 306 [[105, 65, 358, 271]]
image_id: 652 [[284, 273, 511, 483]]
image_id: 55 [[25, 0, 361, 445]]
image_id: 422 [[203, 132, 405, 310]]
image_id: 489 [[376, 0, 825, 307]]
image_id: 323 [[263, 101, 520, 313]]
image_id: 526 [[626, 327, 732, 455], [657, 443, 764, 641]]
image_id: 653 [[196, 95, 347, 377]]
image_id: 97 [[307, 212, 587, 504]]
image_id: 676 [[332, 1, 541, 286]]
image_id: 189 [[299, 344, 372, 438]]
image_id: 440 [[299, 43, 616, 480]]
image_id: 331 [[128, 237, 252, 389], [519, 751, 1099, 1069]]
image_id: 114 [[216, 183, 326, 306]]
image_id: 82 [[0, 91, 21

In [ ]:
# select device (whether GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# DataLoader is iterable over Dataset
for imgs, annotations in val_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)

image_id: 35 [[357, 95, 599, 469]]
image_id:image_id: 59 [[93, 49, 283, 221]]
 254image_id:  [[177, 0, 421, 221]]168
 [[311, 73, 408, 167]]
image_id: image_id:191 image_id: [[186, 153, 428, 402]] 
244image_id: 290 [[215, 152, 410, 392]] 170
[[275, 216, 570, 530]] 
[[137, 159, 443, 477]]
image_id: 67 [[45, 0, 518, 324]]
image_id: 30 [[344, 203, 540, 400]]image_id: 
42 [[402, 181, 622, 528]]image_id: 61 [[189, 113, 537, 529]]

image_id: 163 image_id:[[287, 116, 382, 286]]
 51image_id:  253 [[218, 139, 523, 497], [694, 698, 979, 1012]][[58, 211, 271, 469]]

image_id: 63 [[183, 147, 371, 360]]
image_id: 118 [[150, 2, 542, 500]]
image_id: 166 [[304, 205, 445, 385]]
image_id: 158 [[365, 201, 601, 426]]
image_id: 268 [[280, 103, 458, 188]]
image_id: 261 [[88, 316, 428, 547]]
image_id: 24 [[183, 18, 613, 305]]
image_id: 190 [[248, 71, 428, 228]]
image_id: 232 [[80, 312, 375, 531]]
image_id: 246 [[76, 66, 517, 528]]
image_id: 138 [[219, 131, 607, 528]]
image_id: 247 [[121, 6, 353, 262]]
image_i

In [ ]:
# Run the model on GPU if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load a pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)
_ = model.eval()

print("Model ready")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 151MB/s]


Model ready


In [ ]:
# Download TorchVision repo to use some files from
# references/detection
!pip install pycocotools --quiet
!git clone https://github.com/pytorch/vision.git
!git checkout v0.3.0

!cp vision/references/detection/utils.py ./
!cp vision/references/detection/transforms.py ./
!cp vision/references/detection/coco_eval.py ./
!cp vision/references/detection/engine.py ./
!cp vision/references/detection/coco_utils.py ./

Cloning into 'vision'...
remote: Enumerating objects: 332260, done.
remote: Counting objects: 100% (67170/67170), done.
remote: Compressing objects: 100% (1004/1004), done.
remote: Total 332260 (delta 66698), reused 66510 (delta 66147), pack-reused 265090
Receiving objects: 100% (332260/332260), 671.90 MiB | 18.22 MiB/s, done.
Resolving deltas: 100% (305792/305792), done.
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
'''# # to train on gpu if selected.
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 1

# get the model using our helper function
model = get_object_detection_model(num_classes)

# move model to the right device
model.to(device)
'''
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [ ]:
# Basic python and ML Libraries
import os
import random
import numpy as np
import pandas as pd
# for ignoring warnings
import warnings
warnings.filterwarnings('ignore')

# We will be reading images using OpenCV
import cv2

# xml library for parsing xml files
from xml.etree import ElementTree as et

# matplotlib for visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# torchvision libraries
import torch
import torchvision
from torchvision import transforms as torchtrans
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# these are the helper libraries imported.
from engine import train_one_epoch, evaluate
import utils
import transforms as T

# for image augmentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
# training for 10 epochs
num_epochs = 20

for epoch in range(num_epochs):
    # training for one epoch
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, val_loader, device=device)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
image_id: 391 [[469, 106, 556, 240], [255, 134, 399, 319]]
image_id: 479 [[237, 97, 445, 507]]
image_id: 295 [[330, 125, 381, 184]]
image_id: 481 [[102, 51, 256, 208]]
image_id: 147 [[385, 179, 571, 399], [242, 383, 421, 508]]
image_id: 598 [[248, 42, 520, 352]]
image_id: 341 [[316, 271, 393, 378]]
image_id: 459 [[137, 16, 503, 435]]
image_id: 541 [[90, 129, 444, 361]]
image_id: 580 [[368, 270, 468, 446]]
image_id: 241 [[63, 192, 461, 457]]
image_id: 343 [[295, 486, 1208, 1013]]
image_id: 496 [[309, 151, 899, 792]]
image_id: 303 [[28, 57, 448, 362]]
image_id: 46 [[310, 175, 515, 443]]
image_id: 3 [[187, 14, 543, 526]]
image_id: 308 [[195, 78, 478, 520]]
image_id: 635 [[165, 119, 369, 344]]
image_id: 585 [[421, 111, 540, 396]]
image_id: 410 [[202, 246, 389, 448]]
image_id:  700[[295, 231, 516, 492]]
image_id: 230 [[249, 226, 464, 522]]
image_id: 89 [[344, 36, 604, 285]]
image_id: 277 [[203, 210, 533, 524]]
image_id: 371 [[281, 126, 461, 359]]
image_id

In [ ]:
torch.save(model.state_dict(), 'drive/MyDrive/BSc_Project/medical_faster_rcnn_weight_test.pth')

In [ ]:
# Save the model instance
save_path = "drive/MyDrive/BSc_Project/Code/Models_Medical/medical_faster_rcnn_test"
torch.save(model, save_path)

In [ ]:
model.load_state_dict(torch.load('drive/MyDrive/BSc_Project/medical_faster_rcnn_weight_test.pth'))
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
'''# optimizeer and others
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
criterion = nn.SmoothL1Loss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
len_dataloader = len(data_loader)

## train model
model.train()
num_epochs = 100
best_acc = 0.0
best_epoch = 0
for epoch in range(num_epochs):
    for i, (imgs, annotations) in enumerate(data_loader):
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Iteration: {i}/{len_dataloader}, Loss: {losses}')
    evaluate(model, val_loader, device=device)

'''


"# optimizeer and others\nparams = [p for p in model.parameters() if p.requires_grad]\noptimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)\nlr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)\ncriterion = nn.SmoothL1Loss()\n\ndevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')\nlen_dataloader = len(data_loader)\n\n## train model\nmodel.train()\nnum_epochs = 100\nbest_acc = 0.0\nbest_epoch = 0\nfor epoch in range(num_epochs):\n    for i, (imgs, annotations) in enumerate(data_loader):\n        i += 1\n        imgs = list(img.to(device) for img in imgs)\n        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]\n        loss_dict = model(imgs, annotations)\n        losses = sum(loss for loss in loss_dict.values())\n\n        optimizer.zero_grad()\n        losses.backward()\n        optimizer.step()\n        print(f'Epoch [{epoch+1}/{num_epochs}], Iteration: {i}/{len_dataloader}, Loss: {losses}')\n 